In [ ]:
import os
os.chdir("..")

import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', 2000)
pd.options.display.max_rows = 2000
pd.options.display.max_columns = 200
np.set_printoptions(linewidth=500)
pd.set_option("max_colwidth", 400)

In [ ]:
from PIL import Image
def load_image(image_path: str):
    return Image.open(image_path).convert('RGB')

In [ ]:
from minigpt4.conversation.wrapper import ChatWrapper

In [ ]:
chat = ChatWrapper()

In [ ]:
img = load_image("images/ebay3.jpeg")
img.resize(tuple((np.array(img.size) / 2).astype(int)))

In [ ]:
chat.ask_about_image(
    img,
    "I'd like to sell this product on eBay. Generate a short descriptive title.",
)

In [ ]:
chat.follow_up(
    "Extract the following attributes: color, material, style."
)

In [ ]:
img = load_image("images/anteater.jpeg")
img.resize(tuple((np.array(img.size) / 2).astype(int)))

In [ ]:
chat.ask_about_image(
    img,
    "I'd like to sell this product on eBay. Generate a short descriptive title.",
)

In [ ]:
chat.follow_up(
    "Extract the following attributes: color, material, product type, animal type."
)

In [ ]:
chat.follow_up(
    "It's not an armadillo, it's an anteater."
)

In [ ]:
chat.follow_up(
    "Now, generate a better title."
)

In [ ]:
chat.follow_up(
    "Extract the following attributes: color, material, product type."
)

In [ ]:
chat.follow_up(
    "Extract the following attributes: brand, color, material, theme, product type, occasion, gender, season, style.",
)

In [ ]:
# chat.follow_up(
#     "Extract the following attributes: brand, color, material, theme, product type, product line, character, franchise, year.",
# )

In [ ]:
# chat.follow_up(
#     "In which category should I list it?",
# )

In [ ]:
chat.follow_up(
    "How should I price it?",
)